In [39]:
# Determine number of spots localised within a mask (single image)
#
# -this code runs on a single table of centroid coordinates and binary image mask
# -see the cell below for code that will run on a whole directory
# -the last cell can be used to create outline files for batch processing in FQ


import os
import tifffile
import numpy as np
import pandas as pd

# specify the FQ centroid file
FQ_file = '/Users/joshtitlow/tmp/smFISH_data/localisation_files/20171013_CS_2colorsmFISH_msp670_570_DAPI_HRP405_p1s3r.ome__spots.txt'

# read the FQ file from the metadata header to get pixel size
px_size = pd.read_csv(FQ_file, sep='\t', header=10, nrows=2)
px_xy = px_size['Pix-XY'].iloc[0]
px_z = px_size['Pix-Z'].iloc[0]

# read the FQ file from the data header to get centroid positions
centroid_file = pd.read_csv(FQ_file, sep='\t', header=18)

# convert spatial dimensions to pixel dimensions and store them in separate columns
centroid_file['X_pix'] = np.ceil(centroid_file['Pos_X'].div(float(px_xy)))
centroid_file['Y_pix'] = np.ceil(centroid_file['Pos_Y'].div(float(px_xy)))
centroid_file['Z_pix'] = np.ceil(centroid_file['Pos_Z'].div(float(px_z)))

# create lists to store spot localisation compartment
spot_loc = []

# read image file and show data structure
mask = tifffile.imread('/Users/joshtitlow/tmp/smFISH_data/masks/20171013_CS_2colorsmFISH_msp670_570_DAPI_HRP405_p1s3r.ome.tif')

# index rows in the FQ.csv file to get file names and pixel coordinates
for index, row in centroid_file.iterrows():
    x = row['X_pix'] 
    y = row['Y_pix']
    z = row['Z_pix']
    loc = mask[z, y, x]
    
    # create list that indicates whether each RNA is in cytosol or nucleus 
    if loc == 1:
        spot_loc.append('nucleus')
    if loc == 0:
        spot_loc.append('cytosol')
        
# add RNA localisation list to centroid file
centroid_file['spot_loc'] = spot_loc

# save the updated centroid file
centroid_file.to_csv('test_list.csv', index=False)

//anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:42: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


In [40]:
# Determine number of spots localised within a mask (whole directory)
#
# -this code requires the following sub-directory structure
# -indir (raw data images)
#    -localisation_files (table of centroid coordinates, e.g., FISHQuant __spots.txt files)
#    -masks (binary mask image files)

import os
import tifffile
import numpy as np
import pandas as pd

# specify directory of images
indir = ('/Users/joshtitlow/tmp/smFISH_data/')
infiles = os.listdir(indir)

# create a df and a list to store filename and spot localisations
df = pd.DataFrame()
filename = []
nuc_spots = []
cytosol_spots = []

# analyse each file in the directory
for file in infiles: 
    if file.endswith('.tiff'):
        if not file.startswith('._'):
            
            # specify the FQ centroid file and mask file
            FQ_file = os.path.join(indir, 'localisation_files/', file.replace('.tiff', '__spots.txt'))
            mask = tifffile.imread(os.path.join(indir, 'masks/', file.replace('.tiff', '.tif')))
            
            print 'analysing file:', file
            
            # append filename to list
            filename.append(file)

            # read the FQ file from the metadata header to get pixel size
            px_size = pd.read_csv(FQ_file, sep='\t', header=10, nrows=2)
            px_xy = px_size['Pix-XY'].iloc[0]
            px_z = px_size['Pix-Z'].iloc[0]

            # read the FQ file from the data header to get centroid positions
            centroid_file = pd.read_csv(FQ_file, sep='\t', header=18)

            # convert spatial dimensions to pixel dimensions
            centroid_file['X_pix'] = np.ceil(centroid_file['Pos_X'].div(float(px_xy)))
            centroid_file['Y_pix'] = np.ceil(centroid_file['Pos_Y'].div(float(px_xy)))
            centroid_file['Z_pix'] = np.ceil(centroid_file['Pos_Z'].div(float(px_z)))

            # create lists to store spot localisation compartment
            spot_loc = []
            loc_check = []

            # index rows in the FQ.csv file to get file names and pixel coordinates
            for index, row in centroid_file.iterrows():
                x = row['X_pix'] 
                y = row['Y_pix']
                z = row['Z_pix']
                loc = mask[z, y, x]
                
                #print loc
                if loc == 1:
                    spot_loc.append('nucleus')
                if loc == 0:
                    spot_loc.append('cytosol')

            # write basic statistics to centroid file
            centroid_file['spot_loc'] = spot_loc

            #centroid_file = pd.DataFrame(mean_pixels, columns=["column"])
            #centroid_file.to_csv('test_list.csv', index=False)
                                   
            nuc_spots.append(spot_loc.count('nucleus'))
            cytosol_spots.append(spot_loc.count('cytosol'))
            
df['filename'] = filename
df['nuc_spots'] = nuc_spots
df['cytosol_spots'] = cytosol_spots
df.to_csv('spot_locs.csv', index=False)

analysing file: 20171013_CS_2colorsmFISH_msp670_570_DAPI_HRP405_p1s3r.ome.tiff
analysing file: 20171013_CS_2colorsmFISH_msp670_570_DAPI_HRP405_p1s4l.ome.tiff
analysing file: 20171013_CS_2colorsmFISH_msp670_570_DAPI_HRP405_p1s4r.ome.tiff


//anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:88: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


In [29]:
# specify directory of images
indir = ('/Users/joshtitlow/tmp/smFISH_data/')
infiles = os.listdir(indir)


# requires a template outline file, 
template_file = ('/Users/joshtitlow/tmp/smFISH_data/FQ_outline_template.txt')

# create outline files for batch processing
def FQ_outlines():
    with open(template_file, 'r') as f:
        lines = f.readlines()

    suffix = '__outline.txt'

    for i in infiles:
        if i.endswith('.tiff'):
            if not i.startswith('._'):
                print ('processing %s' % i)
                outname = '%s%s' % (i.split('.ome.tiff')[0], suffix)
                outfile = os.path.join(indir, outname)
                with open(outfile, 'w') as g:
                    newlines = lines[:]
                    fields = newlines[4].split('\t')
                    fields[1] = i
                    thisline = '\t'.join(fields)+'\n'
                    newlines[4] = thisline
                    for line in newlines:
                        g.write(line)
FQ_outlines()

[44, 90, 63]

In [34]:
df

,filename,nuc_spots,cytosol_spots
0,20171013_CS_2colorsmFISH_msp670_570_DAPI_HRP40...,44,547
1,20171013_CS_2colorsmFISH_msp670_570_DAPI_HRP40...,90,442
2,20171013_CS_2colorsmFISH_msp670_570_DAPI_HRP40...,63,396


In [4]:
# Create FQ outline files for batch processing spot detection

import os

# specify directory of images
indir = ('/Users/joshtitlow/tmp/smFISH_data/')
infiles = os.listdir(indir)

template_file = ('/Users/joshtitlow/tmp/smFISH_data/FQ_outline_template.txt')

def FQ_outlines():
    with open(template_file, 'r') as f:
        lines = f.readlines()

    suffix = '__outline.txt'

    for i in infiles:
        if i.endswith('.tiff'):
            if not i.startswith('._'):
                print ('processing %s' % i)
                outname = '%s%s' % (i.split('.ome.tiff')[0], suffix)
                #outname = '%s%s' % (i.split('.tif')[0], suffix)
                outfile = os.path.join(indir, outname)
                with open(outfile, 'w') as g:
                    newlines = lines[:]
                    fields = newlines[4].split('\t')
                    fields[1] = i
                    thisline = '\t'.join(fields)+'\n'
                    newlines[4] = thisline
                    for line in newlines:
                        g.write(line)
FQ_outlines()

processing 20171013_CS_2colorsmFISH_msp670_570_DAPI_HRP405_p1s3r.ome.tiff
processing 20171013_CS_2colorsmFISH_msp670_570_DAPI_HRP405_p1s4l.ome.tiff
processing 20171013_CS_2colorsmFISH_msp670_570_DAPI_HRP405_p1s4r.ome.tiff
